In [ ]:
%matplotlib inline
import numpy as np
from matplotlib.pylab import *

import moldesign as mdt
from moldesign import units as u

<h3>Build a molecule</h3>
Let's start by playing with a simple 2-atom (i.e., diatomic) molecule. In this example, it's set to be two hydrogens, but it doesn't have to be - just make sure there's an even number of electrons. Try, for instance, making HeH+ (make sure to set the charge to 1).

In [ ]:
atom1 = mdt.Atom('H')
atom2 = mdt.Atom('H')
atom2.x = 2.0 * u.angstrom
h2 = mdt.Molecule([atom1,atom2], name='H2', charge=0)
atom1.bond_to(atom2,1)
h2.draw()

<h3>Run quantum chemistry</h3>
Now, we'll use a quantum chemistry model to calculate the electronic state of our molecule. After running the calculation, we'll check out the potential energy and then visualize the molecular orbitals.

In [ ]:
model = mdt.models.PySCFPotential(theory='hf',basis='6-311g*')
h2.set_energy_model(model)
h2.calculate()
print 'Potential energy:', h2.potential_energy.to(u.eV)
h2.draw_orbitals()

In [ ]:
minimization = h2.minimize(frame_interval=1)
minimization.draw_orbitals()

<h3>Scan the potential energy surface</h3>
Now, let's look at how the molecule's energy depends on the distance between the two atoms. First, let's create a series of snapshots of the molecule at different positions:

In [ ]:
distances = np.arange(0.3,7.0,0.2)*u.angstrom
scan = mdt.trajectory.Trajectory(h2)
h2.atoms[0].x = 0.0*u.angstrom
for r in distances:
    h2.atoms[1].x = r
    h2.calc_potential_energy()
    scan.new_frame(annotation='Separation: %s'%r)

Now, we'll plot how the energy and charges on the atoms change with position. For hydrogen, we'll see a potential energy minimum around 0.75 Å - this is the _H<sub>2</sub>_ equilibrium bond length! We can also see, for instance, that the minimum energy at 0.75Å is about 12 eV lower than the energy when we move the atoms far apart - that's the bond energy.

In [ ]:
plot(distances,scan.potential_energy)
grid(); xlabel('separation / ang'); ylabel('energy / eV')
figure()
#plot(distances, scan.mulliken[:,0], label='atom 1 partial charge')
#plot(distances, scan.mulliken[:,1], label='atom 2 partial charge')
grid(); xlabel('separation / ang'); ylabel('partial charge'); legend()

We can also look at what the electronic orbitals are doing as we pull the atoms apart.

In [ ]:
scan.draw_orbitals()

<h3>Something organic</h3>
Let's move on to <a href="https://en.wikipedia.org/wiki/1,3-Butadiene">butadiene</a>, something a little bigger. We'll build the molecule using a <a href="https://en.wikipedia.org/wiki/Simplified_molecular-input_line-entry_system">SMILES string</a> (C=CC=C), associate a model with it, and then minimize its energy.

The visualization, by default, will show you the HOMO orbital. Try setting the "Orbital isovalue" to 0 to see the nodal planes (i.e., the places where the wavefunction value crosses 0).

In [ ]:
mol.draw()

In [ ]:
mol = mdt.from_smiles('C=CC=C')
mol.set_energy_model(mdt.models.RHF(basis='3-21g'))
result = mol.calculate(['potential_energy','orbitals'], wait=True)
print 'Potential energy:',result.potential_energy.to(u.kcalpermol)
mol.draw_orbitals()

Now, let's scan through an internal coordinate, then minimize it.

In [ ]:
mdt.ui.BondSelector(mol)

In [ ]:
traj = mdt.Trajectory(mol)
traj.new_frame()
for angle in np.arange(180, 360.0, 5.0)*u.degrees:
    mdt.set_dihedral(mol.atoms[0], mol.atoms[1], mol.atoms[2], mol.atoms[3], angle)
    mdt.set_dihedral(mol.atoms[1], mol.atoms[2], mol.atoms[3], mol.atoms[9], (angle-180.0*u.degrees)/3.0)
    mol.calculate()
    traj.new_frame()
traj.draw_orbitals()

In [ ]:
mintraj = mol.minimize(nsteps=40, frame_interval=1)

In [ ]:
mintraj = mol.minimize(nsteps=40, frame_interval=1)
newtraj = mdt.Trajectory(mol)
newtraj.frames = traj.frames + mintraj.frames
plot(newtraj.potential_energy); ylabel('energy / eV'); grid()
newtraj.draw_orbitals()

In [ ]:
newtraj.property_keys

# Other Features

### Symmetry
You can use buckyball's symmetrization tools to build symmetric molecules, which can often lead to huge improvements in computational efficiency.

First, let's look at the simplest possible organic molecule, methane. Its simplicity actually leads to some very complex symmetry.

In [ ]:
methane = mdt.from_name('methane')
mdt.symmetry.Symmetrizer(methane)

The list of items on the right is the list of individual symmetry elements - for methane, there are a lot. Click on one to see it illustrated in 3D. Larger molecules will generally have far less symmetry.

Let's take a look at ethane. We'll add some noise to the molecule's coordinates to destroy exact symmetry.

In [ ]:
ethane = mdt.from_smiles('CC')
for atom in ethane.atoms: atom.position += 0.1 * u.angstrom * np.random.random(3)
mdt.symmetry.Symmetrizer(ethane)

You can use the "symmetrize" button to impose individual symmetry elements exactly.

### Introspection
You can dive deeply into the quantum wavefunction results by examining the `h2.electronic_state` object. For instance, we can verify that the MO basis diagonalizes the Fock matrix:

In [ ]:
fock_ao = h2.electronic_state.fock_ao
mos_in_ao = h2.electronic_state.molecular_orbitals

matshow( h2.electronic_state.orbitals.canonical.fock )
title('Fock matrix in canonical MO basis'); cb = colorbar()
cb.set_label('energy / eV')

matshow( h2.electronic_state.orbitals.atomic.fock )
title('Fock matrix in atomic orbital basis'); cb = colorbar()
cb.set_label('energy / eV')